In [6]:
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [11]:
data = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [12]:
data['ORDER_DATE']=pd.to_datetime(data['ORDER_DATE'])

In [13]:
data_grouped = data.groupby(['PRODUCT_ID',data['ORDER_DATE'].dt.to_period('M')]).agg(Total_Order_Qty=('ORDER_QTY','sum')).reset_index()

In [14]:
data_grouped['ORDER_MONTH']=data_grouped['ORDER_DATE'].dt.to_timestamp()

In [15]:
data_grouped = data_grouped[['PRODUCT_ID','ORDER_MONTH','Total_Order_Qty']]

In [16]:
data_grouped = data_grouped.set_index('ORDER_MONTH')

In [17]:
# Group data by 'product_id'
PID_grouped_data = data_grouped.groupby('PRODUCT_ID')

In [18]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_MONTH', 'test_forecast_orders','rse', 'rmse','r2_score'])

In [19]:
forecast_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_MONTH', 'forecast_order_qty'])

In [21]:
for product_id, group in PID_grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)

    print(f'{group}')
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]

    print(f'{train_data.index}')
    
    # Create SARIMAX model
    #model = SARIMAX(train_data['Total_Order_Qty'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

    model = ARIMA(train_data['Total_Order_Qty'], order=(1, 1, 1))  # (p, d, q) parameters
    model_fit = model.fit()
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean

    #print(f'test_df:{test_df.index}')
    
    # Calculate RMSE for order quantity
    #rmse_order_qty = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], forecast_order_qty))
    
    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['Total_Order_Qty'], test_df))
    r2 = r2_score(test_data['Total_Order_Qty'], test_df)

    
    date_list = test_df.index.to_list()
    

    
    #print(f"Order_date: {date_list}")    
    new_row = {
        'PRODUCT_ID': product_id,
        'ORDER_MONTH': date_list,
        'test_forecast_orders': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)

    df_new.set_index('ORDER_MONTH')
    
    test_results=pd.concat([test_results,df_new],ignore_index=True)
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)#, steps=n_forecast)
    forecast_df = forecast.predicted_mean
    print(f'{forecast_df.index}')
    date_list2= forecast_df.index.to_list()
    
    new_row2 = {
        'PRODUCT_ID': product_id,
        'ORDER_MONTH': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    
    forecast_results=pd.concat([forecast_results,df_new2],ignore_index=True)

              PRODUCT_ID  Total_Order_Qty
ORDER_MONTH                              
2019-01-01   Product_001           148400
2019-02-01   Product_001           146650
2019-03-01   Product_001           133150
2019-04-01   Product_001           153600
2019-05-01   Product_001           154650
...                  ...              ...
2024-04-01   Product_001           128300
2024-05-01   Product_001           115350
2024-06-01   Product_001           144950
2024-07-01   Product_001           155700
2024-08-01   Product_001           113300

[68 rows x 2 columns]
DatetimeIndex(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
     

In [22]:
test_results

,PRODUCT_ID,ORDER_MONTH,test_forecast_orders,rse,rmse,r2_score
0,Product_001,2023-07-01,119922.915547,0.0,17133.679442,-0.256974
1,Product_001,2023-08-01,119518.208708,0.0,17133.679442,-0.256974
2,Product_001,2023-09-01,119441.207710,0.0,17133.679442,-0.256974
3,Product_001,2023-10-01,119426.557220,0.0,17133.679442,-0.256974
4,Product_001,2023-11-01,119423.769764,0.0,17133.679442,-0.256974
...,...,...,...,...,...,...
611,Product_100,2024-04-01,25942.448715,0.0,5520.954867,-0.517181
612,Product_100,2024-05-01,25942.448717,0.0,5520.954867,-0.517181
613,Product_100,2024-06-01,25942.448717,0.0,5520.954867,-0.517181
614,Product_100,2024-07-01,25942.448717,0.0,5520.954867,-0.517181
